# Emerging Technologies – Problems

**Author:** Hammad Mubarik  
**Module:** Emerging Technologies  
**Year:** 2025/2026

## Problem 1: Generating Random Boolean Functions

The [Deutsch–Jozsa algorithm](https://learning.quantum.ibm.com/course/fundamentals-of-quantum-algorithms/quantum-query-algorithms#deutschs-algorithm) is a quantum algorithm that figures out whether a given function is constant or balanced in one query. A classical computer would need to check up to half the inputs before it could be sure.

To test the algorithm we need to be able to generate these functions ourselves. The goal here is to write a function called `random_constant_balanced` that returns a random function taking four Boolean inputs and returning a single Boolean output, where the returned function is guaranteed to be either constant or balanced.

- **Constant** means the function returns the same value no matter what you pass in.
- **Balanced** means it returns `True` for exactly half the possible inputs and `False` for the other half.

In [ ]:
# Random selections.
import random

# Numerical arrays and operations.
import numpy as np

# Generating all combinations of inputs.
import itertools as it